In [1]:
import jax
import jax.numpy as jnp
import jax.lax as lax
import jax.random as jrandom
import numpy as np
import optax
import distrax
import haiku as hk

import tensorflow as tf
import tensorflow_datasets as tfds

# TODO: Import NSF from LFIAX
from lfiax.flows.nsf import make_nsf

from typing import Any, Iterator, Mapping, Optional, Sequence, Tuple, Callable, Union

Array = jnp.ndarray
PRNGKey = Array
Batch = Mapping[str, np.ndarray]
OptState = Any

def sim_linear_jax(d: Array, priors: Array, key: PRNGKey):
    # Keys for the appropriate functions
    keys = jrandom.split(key, 3)

    # sample random normal dist
    noise_shape = (1,)

    mu_noise = jnp.zeros(noise_shape)
    sigma_noise = jnp.ones(noise_shape)
    
    n_n = distrax.Independent(
          distrax.MultivariateNormalDiag(mu_noise, sigma_noise)).sample(
              seed=keys[0],
              sample_shape=[len(d),
                            len(priors)])
    
    # sample random gamma noise
    n_g = distrax.Gamma(2., 1./2.).sample(seed=keys[1], 
                                          sample_shape=[len(d),
                                                        len(priors)
                                                        ]
                                          )
    
    # perform forward pass
    y = jnp.broadcast_to(priors[:,0], (len(d), len(priors)))
    y = y + jnp.expand_dims(d, 1) @ jnp.expand_dims(priors[:,1], 0) 
    y = y + n_g + jnp.squeeze(n_n)
    ygrads = priors[:,1]

    return y, ygrads

# ----------------------------------------
# Helper functions to simulate data
# ----------------------------------------
def load_dataset(split: tfds.Split, batch_size: int) -> Iterator[Batch]:
  ds = split
  ds = ds.shuffle(buffer_size=10 * batch_size)
  ds = ds.batch(batch_size)
  ds = ds.prefetch(buffer_size=1000)
  ds = ds.repeat()
  return iter(tfds.as_numpy(ds))


def sim_data(d: Array, priors: Array, key: PRNGKey):
  '''
  Returns data in a format suitable for normalizing flow training.
  Data will be in shape [y, thetas]. The `y` variable can vary in size.
  '''
  keys = jrandom.split(key, 2)
  
  theta_shape = (2,)

  mu = jnp.zeros(theta_shape)
  sigma = (3 ** 2) * jnp.ones(theta_shape) 

  base_distribution = distrax.Independent( # Should this be independent?
        distrax.MultivariateNormalDiag(mu, sigma))

  priors = base_distribution.sample(seed=keys[0], sample_shape=[num_samples])
  
  y, ygrads = sim_linear_jax(d, priors, keys[1])
  
  return jnp.column_stack( (y.T, jnp.squeeze(priors)) )


def prepare_data(batch: Batch, prng_key: Optional[PRNGKey] = None) -> Array:
  data = batch.astype(np.float32)
  # Handling the scalar case
  if data.shape[1] <= 3:
      x = jnp.expand_dims(data[:, :-2], -1)
  x = data[:, :-2]
  thetas = data[:, -2:]
  return x, thetas

# ----------------------------
# Haiku transform functions for training and evaluation
# ----------------------------
@hk.without_apply_rng
@hk.transform
def log_prob(data: Array, cond_data: Array) -> Array:
  # Get batch 
  shift = data.mean(axis=0)
  scale = data.std(axis=0) + 1e-14
  
  model = make_nsf(
      event_shape=EVENT_SHAPE,
      cond_info_shape=cond_info_shape,
      num_layers=flow_num_layers,
      hidden_sizes=[hidden_size] * mlp_num_layers,
      num_bins=num_bins,
      standardize=True,
      shift=shift,
      scale=scale)
  return model.log_prob(data, cond_data)

@hk.without_apply_rng
@hk.transform
def model_sample(key: PRNGKey, num_samples: int, cond_data: Array) -> Array:
  model = make_nsf(
      event_shape=EVENT_SHAPE,
      cond_info_shape=cond_info_shape,
      num_layers=flow_num_layers,
      hidden_sizes=[hidden_size] * mlp_num_layers,
      num_bins=num_bins)
  z = jnp.repeat(cond_data, num_samples, axis=0)
  z = jnp.expand_dims(z, -1)
  return model._sample_n(key=key, 
                         n=[num_samples],
                         z=z)

def loss_fn(params: hk.Params, prng_key: PRNGKey, batch: Batch) -> Array:
  x, thetas = prepare_data(batch, prng_key)
  # Loss is average negative log likelihood.
  loss = -jnp.mean(log_prob.apply(params, x, thetas))
  return loss


@jax.jit
def eval_fn(params: hk.Params, batch: Batch) -> Array:
  x, thetas = prepare_data(batch) 
  loss = -jnp.mean(log_prob.apply(params, x, thetas))
  return loss


@jax.jit
def update(params: hk.Params,
            prng_key: PRNGKey,
            opt_state: OptState,
            batch: Batch) -> Tuple[hk.Params, OptState]:
  """Single SGD update step."""
  grads = jax.grad(loss_fn)(params, prng_key, batch)
  updates, new_opt_state = optimizer.update(grads, opt_state)
  new_params = optax.apply_updates(params, updates)
  return new_params, new_opt_state

In [3]:
seed = 1231
key = jrandom.PRNGKey(seed)

# d = jnp.array([-10.,0., 5., 10.])
# d = jnp.array([1., 2.])
d = jnp.array([1.])
num_samples = 100

# Params and hyperparams
theta_shape = (2,)
EVENT_SHAPE = (len(d),) 
cond_info_shape = theta_shape

batch_size = 128
flow_num_layers = 10
mlp_num_layers = 4
hidden_size = 500
num_bins = 4
learning_rate = 1e-4

# using 100,000 steps could take long (about 2 hours) but will give better results. 
# You can try with 10,000 steps to run it fast but result may not be very good

training_steps =  10#00
eval_frequency =  100

optimizer = optax.adam(learning_rate)

# Simulating the data to be used to train the flow.
num_samples = 10000
X = sim_data(d, num_samples, key)

# Create tf dataset from sklearn dataset
dataset = tf.data.Dataset.from_tensor_slices(X)

# Splitting into train/validate ds
train = dataset.skip(2000)
val = dataset.take(2000)

# load_dataset(split: tfds.Split, batch_size: int)
train_ds = load_dataset(train, 512)
valid_ds = load_dataset(val, 512)

# Training
prng_seq = hk.PRNGSequence(42)
params = log_prob.init(next(prng_seq), 
                    np.zeros((1, *EVENT_SHAPE)), 
                    np.zeros((1, *cond_info_shape)))
opt_state = optimizer.init(params)


for step in range(training_steps):
    params, opt_state = update(params, next(prng_seq), opt_state,
                                next(train_ds))

    if step % eval_frequency == 0:
        val_loss = eval_fn(params, next(valid_ds))
        print(f"STEP: {step:5d}; Validation loss: {val_loss:.3f}")

/Users/vincent_zaballa/anaconda3/envs/sdm3/lib/python3.9/site-packages/haiku/_src/base.py:515: UserWarning: Explicitly requested dtype float64 requested in zeros is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  param = init(shape, dtype)


STEP:     0; Validation loss: 3.980
